# Analysis of Belle2 MonteCarlo Data

__Authors:__

- Valeria Fioroni (University of Padova)
- Matteo Guida (University of Padova)
- Philipp Zehetner (University of Padova, Ludwig Maximilian University of Munich)
- [Stack Overflow](https://stackoverflow.com/)


In [1]:
import os, sys, pprint
import glob
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
Testing = True

## Reading and Preparing Data

In [3]:
# Define branches that we need for our analysis
Masses = ['B0_M', 'B0_ErrM', 'B0_SigM', 'B0_K_S0_M', 'B0_K_S0_ErrM', 'B0_K_S0_SigM', 'B0_etap_M', 'B0_etap_ErrM', 
          'B0_etap_SigM', 'B0_etap_eta_M', 'B0_etap_eta_ErrM', 'B0_etap_eta_SigM']
Kinetics_CMS = ['B0_Pcms', 'B0_etap_Pcms', 'B0_etap_eta_Pcms', 'B0_etap_eta_gamma0_Pcms', 'B0_etap_eta_gamma1_Pcms',
                'B0_etap_pi0_Pcms', 'B0_etap_pi1_Pcms', 'B0_K_S0_Pcms']
Other_Kinetics = ['B0_deltae', 'B0_mbc']
DecayAngles = ['B0_decayAngle__bo0__bc', 'B0_decayAngle__bo1__bc', 'B0_etap_decayAngle__bo0__bc',
               'B0_etap_decayAngle__bo1__bc', 'B0_etap_decayAngle__bo2__bc']
Positions = ['B0_X', 'B0_ErrX', 'B0_Y', 'B0_ErrY', 'B0_Z', 'B0_ErrZ', 'B0_Rho',  
             'B0_etap_X', 'B0_etap_ErrX', 'B0_etap_Y', 'B0_etap_ErrY', 
             'B0_etap_Z', 'B0_etap_ErrZ', 'B0_etap_Rho',
             'B0_etap_eta_X', 'B0_etap_eta_ErrX', 'B0_etap_eta_Y',
             'B0_etap_eta_ErrY', 'B0_etap_eta_Z', 'B0_etap_eta_ErrZ', 'B0_etap_eta_Rho',
             'B0_etap_pi0_X', 'B0_etap_pi0_ErrX', 'B0_etap_pi0_Y', 'B0_etap_pi0_ErrY', 
             'B0_etap_pi0_Z', 'B0_etap_pi0_ErrZ', 'B0_etap_pi0_Rho', 
             'B0_etap_pi1_X', 'B0_etap_pi1_ErrX', 'B0_etap_pi1_Y', 'B0_etap_pi1_ErrY', 
             'B0_etap_pi1_Z', 'B0_etap_pi1_ErrZ', 'B0_etap_pi1_Rho', 
             'B0_K_S0_X', 'B0_K_S0_ErrX', 'B0_K_S0_Y', 'B0_K_S0_ErrY', 'B0_K_S0_Z',
             'B0_K_S0_ErrZ', 'B0_K_S0_Rho', 
             'B0_cosAngleBetweenMomentumAndVertexVector', 'B0_distance', 'B0_significanceOfDistance',
             'B0_dr', 'B0_etap_pi0_dr', 'B0_etap_pi1_dr', 'B0_K_S0_dr']
Vertex_Training = ['B0_VtxPvalue', 'B0_etap_VtxPvalue', 'B0_etap_eta_VtxPvalue', 'B0_etap_pi0_VtxPvalue',
                   'B0_etap_pi1_VtxPvalue', 'B0_K_S0_VtxPvalue', ]
Continuum_Suppression_Training = ['B0_TrCSMVA']

Training = Kinetics_CMS + Masses + Other_Kinetics + Continuum_Suppression_Training + Positions + DecayAngles + Vertex_Training
Important = Training + ['B0_isSignal']

In [4]:
from root_pandas import read_root

path = '/home/philipp/Desktop/Project/DATA/'
SFiles = glob.glob(os.path.join(path, 'Signal/*.root'))
CFiles = glob.glob(os.path.join(path, 'Continuous/*.root'))
PFiles = glob.glob(os.path.join(path, 'Peaking/*.root'))

Full_Signal = pd.concat((read_root(f, 'B0', columns=Important) for f in SFiles))
Full_Signal = Full_Signal[Full_Signal['B0_isSignal']==1].reset_index(drop=True)
Full_Continuous = pd.concat((read_root(f, 'B0', columns=Important) for f in CFiles))
Full_Peaking = pd.concat((read_root(f, 'B0', columns=Important) for f in PFiles))

Signal = Full_Signal[Training]
Continuous = Full_Continuous[Training]
Peaking = Full_Peaking[Training]

Welcome to JupyROOT 6.16/00


/home/philipp/anaconda3/envs/Belle/lib/python3.7/site-packages/root_numpy/__init__.py:46: RuntimeWarning: numpy 1.16.4 is currently installed but you installed root_numpy against numpy 1.9.3. Please consider reinstalling root_numpy for this numpy version.
  RuntimeWarning)


In [5]:
Signal['Type'] = 2
Continuous['Type'] = 1
Peaking['Type'] = 0

## Preprocessing

In [6]:
from random import seed
from random import randint

n_seed=1234
seed(n_seed)

In [7]:
from sklearn.model_selection import train_test_split

Sum_BS = pd.concat([Signal, Continuous, Peaking]).sample(frac=1)
X = Sum_BS.drop('Type',axis=1)
Y = Sum_BS['Type']

# For Testing purposes it's convenient to work with a small subset
if Testing : 
    N_Events = 10000
    X = X[:N_Events]
    Y = Y[:N_Events]

# We split our dataset into 70% training, 15% testing and 15% validation 
X_Train, X_Rest, Y_Train, Y_Rest = train_test_split(X, Y, train_size=0.7,random_state=randint(10**6,10**9))
X_Test, X_Validation, Y_Test, Y_Validation = train_test_split(X_Rest, Y_Rest, train_size=0.5,random_state=randint(10**6,10**9))

In [8]:
from sklearn.preprocessing import StandardScaler
from keras.utils import to_categorical

ss = StandardScaler()
X_Train = ss.fit_transform(X_Train)
X_Test = ss.transform(X_Test)
X_Validation = ss.transform(X_Validation)

Y_Train = to_categorical(Y_Train, num_classes=3) 
Y_Test = to_categorical(Y_Test, num_classes=3)
Y_Validation = to_categorical(Y_Validation, num_classes=3)

Using TensorFlow backend.


In [9]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

pca = PCA(n_components=len(Training))
pcTrain = pd.DataFrame(data = pca.fit_transform(X_Train))
pcTest = pd.DataFrame(data = pca.transform(X_Test))
pcValidation = pd.DataFrame(data = pca.transform(X_Validation))

T50, T90, T95, T99, = False, False, False, False
for i in range(len(pca.explained_variance_ratio_)) : 
    if (sum(pca.explained_variance_ratio_[:i+1]) > 0.5 and T50 ==False) : 
        print(str(i+1) + ' variables explain 50% of the variance')
        T50 = True
        n_50 = i + 1 
    if (sum(pca.explained_variance_ratio_[:i+1]) > 0.9 and T90 ==False) : 
        print(str(i+1) + ' variables explain 90% of the variance')
        T90 = True
        n_90 = i + 1
    if (sum(pca.explained_variance_ratio_[:i+1]) > 0.95 and T95 ==False) : 
        print(str(i+1) + ' variables explain 95% of the variance')
        T95 = True
        n_95 = i + 1
    if (sum(pca.explained_variance_ratio_[:i+1]) > 0.99 and T99 ==False) : 
        print(str(i+1) + ' variables explain 99% of the variance')
        T99 = True
        n_99 = i + 1
        
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Number of Components')
plt.ylabel('Explained Variance')
plt.show()

11 variables explain 50% of the variance
35 variables explain 90% of the variance
41 variables explain 95% of the variance
50 variables explain 99% of the variance


<Figure size 640x480 with 1 Axes>

In [10]:
# For input_dim chose n_50, n_90, n_95 or n_99 depending on how much explained variance we require
input_dim = n_99
pcTrain = pcTrain.iloc[:, 0:input_dim]
pcTest = pcTest.iloc[:, 0:input_dim]
pcValidation = pcValidation.iloc[:, 0:input_dim]

## Create the Neural Network 

In [17]:
optimizer = ['SGD', 'Adam']
epochs = [10, 30]
epochs = [1, 2]
batch_size = [100, 1000]
architectures = [ [50, 50], [50, 100, 50], [50, 100, 100, 50] ] 
activation = ['relu', 'elu']

In [31]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.constraints import maxnorm
from keras.wrappers.scikit_learn import KerasClassifier

def build_DNN(optimizer='adam', activation='relu'):
    model = Sequential()
    model.add(Dropout(0.2, input_shape=(pcTrain.shape[1],)))
    model.add(Dense(layers[0], input_shape=(pcTrain.shape[1],), activation='relu'))
    for i in range(1,len(layers)):
        model.add(Dense(layers[i], activation='relu', kernel_constraint=maxnorm(3)))
    model.add(Dense(3, activation='sigmoid'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def print_results() :
    print("Layers: ", layers)
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params): 
        print("%f (%f) with %r" % (mean, stdev, param))


It seems as the GridSearch does not provide the possibility to try out different architectures. 
As a workaround I suggest defining layers=[xx, xx, xx] for a couple of different architectures. 

In [32]:
from sklearn.model_selection import GridSearchCV

results = []
grids = []
for architecture in architectures : 
    layers = architecture
    print("Using architecture: ", layers)
    model = KerasClassifier(build_fn=build_DNN, batch_size=10000, epochs=2)
    param_grid = dict(epochs=epochs, batch_size=batch_size,
                      activation=activation, optimizer=optimizer)
    grid = GridSearchCV(estimator=model, param_grid=param_grid,
                        n_jobs=-1, pre_dispatch=8)
    grid_result = grid.fit(pcTrain, Y_Train)
    results.append(grid_result)
    grids.append(grid)
    print_results()

Using architecture:  [50, 50]


/home/philipp/anaconda3/envs/Belle/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
W0714 19:51:26.817554 140236883281728 deprecation_wrapper.py:119] From /home/philipp/anaconda3/envs/Belle/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0714 19:51:26.835836 140236883281728 deprecation_wrapper.py:119] From /home/philipp/anaconda3/envs/Belle/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0714 19:51:26.838609 140236883281728 deprecation_wrapper.py:119] From /home/philipp/anaconda3/envs/Belle/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placehold

Epoch 1/2
7000/7000 [==============================] - 1s 93us/step - loss: 0.7465 - acc: 0.6809
Epoch 2/2
7000/7000 [==============================] - 0s 20us/step - loss: 0.2830 - acc: 0.9029
Layers:  [50, 50]
Best: 0.958143 using {'activation': 'elu', 'batch_size': 100, 'epochs': 2, 'optimizer': 'SGD'}
0.891857 (0.031474) with {'activation': 'relu', 'batch_size': 100, 'epochs': 1, 'optimizer': 'SGD'}
0.914143 (0.002961) with {'activation': 'relu', 'batch_size': 100, 'epochs': 1, 'optimizer': 'Adam'}
0.957000 (0.005075) with {'activation': 'relu', 'batch_size': 100, 'epochs': 2, 'optimizer': 'SGD'}
0.955143 (0.005682) with {'activation': 'relu', 'batch_size': 100, 'epochs': 2, 'optimizer': 'Adam'}
0.600143 (0.026727) with {'activation': 'relu', 'batch_size': 1000, 'epochs': 1, 'optimizer': 'SGD'}
0.506143 (0.137326) with {'activation': 'relu', 'batch_size': 1000, 'epochs': 1, 'optimizer': 'Adam'}
0.679000 (0.060339) with {'activation': 'relu', 'batch_size': 1000, 'epochs': 2, 'optimi

2333/2333 [==============================] - 1s 316us/step
Epoch 1/2
4667/4667 [==============================] - 2s 455us/step - loss: 0.6597 - acc: 0.7191
Epoch 2/2
2333/2333 [==============================] - 1s 298us/step
Epoch 1/2
4666/4666 [==============================] - 2s 447us/step - loss: 0.6042 - acc: 0.7392
Epoch 2/2
2334/2334 [==============================] - 1s 302us/step
Epoch 1/2
4667/4667 [==============================] - 2s 504us/step - loss: 0.6210 - acc: 0.6662
Epoch 2/2
2333/2333 [==============================] - 1s 367us/step
Epoch 1/2
4667/4667 [==============================] - 2s 518us/step - loss: 0.7045 - acc: 0.6531
Epoch 2/2
2333/2333 [==============================] - 1s 337us/step
Epoch 1/1
2334/2334 [==============================] - 1s 327us/step
Epoch 1/1
2333/2333 [==============================] - 1s 330us/step
Epoch 1/1
2333/2333 [==============================] - 1s 351us/step
Epoch 1/1
2334/2334 [==============================] - 1s 347us/st

Epoch 1/1
2333/2333 [==============================] - 1s 559us/step
Epoch 1/1
2333/2333 [==============================] - 1s 579us/step
Epoch 1/1
2334/2334 [==============================] - 1s 596us/step
Epoch 1/1
2333/2333 [==============================] - 1s 583us/step
Epoch 1/1
2333/2333 [==============================] - 1s 604us/step
Epoch 1/2
4666/4666 [==============================] - 4s 807us/step - loss: 1.0227 - acc: 0.5150
Epoch 2/2
2334/2334 [==============================] - 1s 610us/step
Epoch 1/2
4667/4667 [==============================] - 4s 826us/step - loss: 1.0550 - acc: 0.4553
Epoch 2/2
2333/2333 [==============================] - 1s 629us/step
Epoch 1/2
4667/4667 [==============================] - 4s 840us/step - loss: 1.0918 - acc: 0.3739
Epoch 2/2
2333/2333 [==============================] - 2s 650us/step
Epoch 1/2
4666/4666 [==============================] - 4s 846us/step - loss: 1.0923 - acc: 0.3506
Epoch 2/2
2334/2334 [==============================] - 2

Using TensorFlow backend.
W0714 19:50:31.310363 140586342672192 deprecation_wrapper.py:119] From /home/philipp/anaconda3/envs/Belle/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0714 19:50:31.324429 140586342672192 deprecation_wrapper.py:119] From /home/philipp/anaconda3/envs/Belle/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0714 19:50:31.325693 140586342672192 deprecation_wrapper.py:119] From /home/philipp/anaconda3/envs/Belle/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0714 19:50:31.333215 140586342672192 deprecation.py:506] From /home/philipp/anaconda3/envs/Belle/lib/python3.7/site-packages/keras/backend/tensorflow_backen

In [37]:
for result in results : 
    pprint.pprint(result.best_score_)
    pprint.pprint(result.best_params_)

0.9581428618771689
{'activation': 'elu', 'batch_size': 100, 'epochs': 2, 'optimizer': 'SGD'}
0.9597142886860031
{'activation': 'elu', 'batch_size': 100, 'epochs': 2, 'optimizer': 'SGD'}
0.9624285731996809
{'activation': 'elu', 'batch_size': 100, 'epochs': 2, 'optimizer': 'SGD'}
